In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import traceback

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("vinai/vinai-translate-vi2en", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-vi2en")
device_vi2en = torch.device("cuda")
model_vi2en.to(device_vi2en)


def translate_vi2en(vi_texts: str) -> str:
    try:
        input_ids = tokenizer_vi2en(vi_texts, padding=True, return_tensors="pt").to(device_vi2en)
        output_ids = model_vi2en.generate(
            **input_ids,
            decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
            num_return_sequences=1,
            num_beams=5,
            early_stopping=True
        )
        en_texts = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
        return en_texts
    except: 
        # print full error
        print("Error: ", sys.exc_info()[0])
        # print trace back
        print(traceback.format_exc())
        # print error line
        print("Error: ", vi_texts)

# # The input may consist of multiple text sequences, with the number of text sequences in the input ranging from 1 up to 8, 16, 32, or even higher, depending on the GPU memory.
# vi_texts = ["Cô cho biết: trước giờ tôi không đến phòng tập công cộng, mà tập cùng giáo viên Yoga riêng hoặc tự tập ở nhà.",
#             "Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn.",
#             "cô cho biết trước giờ tôi không đến phòng tập công cộng mà tập cùng giáo viên yoga riêng hoặc tự tập ở nhà khi tập thể dục trong không gian riêng tư tôi thoải mái dễ chịu hơn"]
# print(translate_vi2en(vi_texts))

/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
vn3k = pd.read_json('/home/phongtnh/Person-Search/vn3k.json')
vn3k = vn3k[['file_path', 'captions']]

In [7]:
# tracking progression using tqdm
from tqdm import tqdm
tqdm.pandas()

vn3k['vi_captions'] = vn3k['captions'].progress_apply(translate_vi2en)

  0%|          | 0/6302 [00:00<?, ?it/s]/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  2%|▏         | 112/6302 [00:50<34:36,  2.98it/s] 

Error:  <class 'IndexError'>
Traceback (most recent call last):
  File "/tmp/ipykernel_3106436/2079250303.py", line 12, in translate_vi2en
    input_ids = tokenizer_vi2en(vi_texts, padding=True, return_tensors="pt").to(device_vi2en)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2577, in __call__
    encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2663, in _call_one
    return self.batch_encode_plus(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/phongtnh/miniconda3/envs/person_search/lib/python3.11/site-packages/transformers/tokenization_utils_base.py", line 2854, in batch_encode_plus
    return self

100%|██████████| 6302/6302 [42:18<00:00,  2.48it/s]  


In [8]:
vn3k.to_csv('vn3k.csv')

In [10]:
vn3k[['captions', 'vi_captions']].to_csv('vn3k_trans_review.csv')